# Running a Modelica model in a conda package
This example illustrates how to test a Modelica model from a conda package in a fresh environment `packtest` that has been created e.g. through
```
conda create -n packtest python=3.8 jupyterlab pyarrow matplotlib pymodelicademo -c ./build
```
See https://openmodelica.org/doc/OpenModelicaUsersGuide/v1.11.0/ompython.html

In [ ]:
import pandas as  pd
import os
import sys
import json
import tempfile
import importlib

In [ ]:
import DyMat

## Parametric simulation without OMC
https://openmodelica.org/doc/OpenModelicaUsersGuide/latest/scripting_api.html#simulation-parameter-sweep

In [ ]:
from ModelicaModels import BouncingBall

In [ ]:
parameters_var_df = pd.DataFrame(columns=['run_key', 'modifiers'], data=[
    ['r1', {'e': 0.7, 'stopTime': 5.0}],
    ['r2', {'e': 0.5, 'stopTime': 5.0}],
    ['r3', {'e': 0.9, 'stopTime': 5.0}],
])
parameters_var_df['modifiers'] = parameters_var_df['modifiers'].apply(json.dumps)
parameters_var_df

### Sequential execution

In [ ]:
mod = BouncingBall.instantiatemodel('BouncingBall')

In [ ]:
mod.getParameters()

In [ ]:
from ModelicaRuntimeTools import get_sim_dist_func

In [ ]:
%%time
ts_all_df = parameters_var_df.groupby(['run_key']).apply(
        get_sim_dist_func(BouncingBall, 'BouncingBall', res_vars=['h', 'v'])
    )

In [ ]:
ts_all_df

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
p_df = ts_all_df[[
    'time', 'run_key', 'h'
]].reset_index(drop=True).drop_duplicates(subset=['time', 'run_key'])
p_df.pivot(index='time',columns='run_key', values='h').plot()

### Parallel execution with Spark

In [ ]:
parameters_var_sdf = spark.createDataFrame(parameters_var_df)
parameters_var_sdf.show()

### Parallel execution

In [ ]:
from pyspark.sql import types as T, functions as F
res_schema = T.StructType([
    T.StructField("time", T.DoubleType(), True),
    T.StructField("h", T.DoubleType(), True),
    T.StructField("v", T.DoubleType(), True),
    T.StructField("run_key", T.StringType(), True),
])
# Running the parametric simulation
ts_sim_sdf = parameters_var_sdf.groupby(['run_key']).applyInPandas(
        get_sim_dist_func(BouncingBall, 'BouncingBall', res_vars=['h', 'v'], use_local=True), schema=res_schema,
    ).cache()

In [ ]:
ts_sim_sdf.show()

In [ ]:
for n, run_key in enumerate(parameters_var_df['run_key'].unique()):
    ts_run_sim_pdf = ts_sim_sdf.where(F.col('run_key') == run_key).toPandas()
    plt.plot(ts_run_sim_pdf['time'], ts_run_sim_pdf['h'])
plt.show()